In [247]:
import time
import requests
import json
import logging
from datetime import datetime
from elasticsearch import Elasticsearch

### Connection elastic search

In [206]:
# test le port elasticsearch
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('ElasticSearch est bien connecté')
    else:
        print('Pas de réponse')
    return _es

In [207]:
# crée une connection elasticsearch
es = connect_elasticsearch()

ElasticSearch est bien connecté


In [208]:
# infos connection
dir(es)
es.info()

{'name': '8be590a5377a',
 'cluster_name': 'cluster-vincent',
 'cluster_uuid': 'wr35sY1jR9CRZD2MgtWhjQ',
 'version': {'number': '7.9.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91',
  'build_date': '2020-09-01T21:22:21.964974Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### Initialisation de l'index et mapping
Méthodologie à adopter pour `init_process` :
* True : si on crée un nouvel index
* False : si on met à jour directement l'index existant

In [299]:
init_process = True

In [300]:
# nom de l'index
index_name = "python_traffic"

In [301]:
#if init_process:

In [210]:
# vérifie si l'index existe ou pas
if es.indices.exists(index=index_name):
    print("Index supprimé:", index_name)
    es.indices.delete(index=index_name)
else:
    print("Index non existant")

Index supprimé: python_traffic


In [209]:
# mappings pour les coordonnées geospatiales
req_body = {
    "mappings": {
        "properties": {
            "fields": {
                "properties": {
                    # fields.geo_point_2d
                    "geo_point_2d": { "type":"geo_point" },
                    
                    # fields.geo_shape
                    "geo_shape": { "type":"geo_shape" }
                }
            },
            
            "geometry": {
                "properties": {
                    # geometry.coordinates
                    "coordinates": { "type":"geo_point" }
                 }
             }
        }
    }
}

In [211]:
# crée l'índex vide
es.indices.create(index=index_name, body=req_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python_traffic'}

### Import depuis l'api traffic

In [288]:
# url
nb_rows = 100
traffic_url = "https://data.rennesmetropole.fr/api/records/1.0/search/?dataset=etat-du-trafic-en-temps-reel&q=&rows="+str(nb_rows)

In [289]:
# get
traffic = requests.get(traffic_url)
if traffic.status_code==200:
    print("ok!")

ok!


In [290]:
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [291]:
#jprint(traffic.json())

In [292]:
# les données
data_json = traffic.json()
data = data_json["records"]
nb_rows2 = len(data)
print("Nombre de document importés:", nb_rows2)

Nombre de document importés: 100


### Data processing

In [293]:
#data[0]["fields"]["traveltimereliability"]

In [294]:
# reliability >50%
data = [i for i in data if i["fields"]["traveltimereliability"]>=0]

In [295]:
nb_rows3 = len(data)
print("Nombre de document supprimés:", nb_rows2-nb_rows3)

Nombre de document supprimés: 0


### Export vers elasticsearch

In [296]:
# export vers elastic
i = es.count(index=index_name)['count']+1
j = 0
for doc_body in data:
    req = es.index(index=index_name, id=i, doc_type='_doc', body=doc_body)
    i = i+1
    j = j+1
print("Nombre de document exportés:", j)

Nombre de document exportés: 100


In [297]:
time.sleep(3)
print("Nombre de document total:", es.count(index=index_name)["count"])

Nombre de document total: 1410


In [ ]:
#es.get(index=index_name, doc_type='_doc', id=4)

Note :
les étapes 'import', 'data processing' et 'export' sont à placer dans un batch et à refaire en boucle.

----